**Exercise 1 [The SCS problem].**
Let $\Sigma = \{0, 1\}$ be a finite alphabet.
The *shortest common substring* problem (SCS) has as input a set of finite strings
$S = \{w_1, \dots, w_m\}$ with $w_i \in \Sigma^*$
and a bound $n$, and asks to determine whether there is a string $w \in \Sigma^n$ of length $n$
s.t. every $w_i$ is a (contiguous) substring of $w$. The SCS problem is NP-complete.
1. Encode the SCS problem as a SAT problem and use Z3 to solve it.
2. Can you reconstruct a solution $w$ from the Z3 model?

In [2]:
from z3 import *

# input example
S = ["001010011101", "1001010101011", "101101010101010",
     "10111011001010101010", "10110101011010101010", "10110101101010101010"]
m = len(S)
n = 75

# how to declare a list of n variables x0, ..., x(n-1)
x = [ Bool('x' + str(k)) for k in range(n) ]

# Solution for S: 001010011101101011010101010100101010101110110010101010101101010110101010100

# this procedure returns an assignment (as a dictionary) if any exists
def model(phi):
    
    # create a SAT instance
    s = Solver()
    s.add(phi)

    # return a satisfying assignment
    return s.model() if s.check() == sat else []

def formula():
    # Check if all words are present as substrings of result
    substrs = []
    for word in S:
        # Check if word occurs in resulting string
        subform = []
        for i in range(n - len(word)):
            # Check if word is on exact position of result
            pos = []
            for j in range(len(word)):
                # Check if one letter of a word is in exact position of result
                var = x[i + j] if word[j] == '1' else Not(x[i + j])
                pos.append(var)
            subform.append(And(*pos))
        substrs.append(Or(*subform))
    formula = And(*substrs)
    
    return formula

def printres(model):
    res = ""
    for i in range(n):
        v = '1' if model[x[i]] else '0'
        res += v
    
    for w in S:
        if w not in res:
            print "Error!!!"
    
    print res
    

# Example
phi = formula()
result = model(phi)
printres(result)

001010011101101011010101010100101010101110110010101010101101010110101010100


**Exercise 2 [Phase transition].**
In this exercise we explore the phenomenon of *phase transition* for the $k$-SAT problem.
Let $n$ be the number of variables $X = \{x_1, \dots, x_n\}$ and let $m$ be the number of clauses.
A random $k$-SAT instance is obtained by producing each of the $m$ clauses according to the following process:

* Extract $k$ variables without replacement from $X$ and determine independently and uniformly whether each variable appears positively or negatively.

Fix $n = 100$, $k = 3$,
and let $p_{m/n}$ be the probability that a SAT instance randomly generated as above is satisfiable.
1. For a fixed $m$, compute an estimate $\hat p_{m / n}$ by sampling $N = 100$ instances as above.
2. Plot the estimates $\hat p_{m / n}$ as a function of $m \in \{1, \dots, 10 * n \}$.
3. What is an "interesting" interval for $m$? Refine the interval above if necessary.

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random

# number of variables
n = 10

# size of a clause
k = 3

# number of samples per point
N = 100

variables = [Bool('x' + str(i)) for i in range(n)]

def get_instance(m):
    clauses = []
    for i in range(m):
        vs = np.random.choice(variables, k, replace=False)
        
        vals = []
        for v in vs:
            fi = v if random.randint(0, 1) == 0 else Not(v)
            vals.append(fi)
        
        clauses.append(Or(*vals))
    return And(*clauses)

def estimate(m):
    satisfied = []
    
    for i in range(N):
        phi = get_instance(m)
        
        s = Solver()
        s.add(phi)
        result = 1 if s.check() == sat else 0
        satisfied.append(result)
    
    return np.mean(satisfied)

estimate(3)

def makeplot():
    values = []
    for m in range(10 * n):
        values.append(estimate(m))
    plt.figure()
    plt.plot(values)
    plt.show()

#makeplot()

# Hints:
# np.random.choice(list) returns a random element from list
# np.random.choice(list, k, replace=False) returns k random elements from list *without replacement*
# np.mean(list) computes the average of number in list
# plt.plot(list) generates a plot from a list of values
# plt.show() displays the plot

**Exercise 3 [Factoring].**
It is common wisdom that satisfiable SAT instances are easier to decide than unsatisfiable ones. In this exercise we will explore methods to produce hard *satisfiable* instances.
The idea is to take a product of two large primes $n = p * q$
and to write a SAT instance which satisfied iff there are $a, b > 1$ s.t. $a * b = n$.
By construction, this is a satisfiable instance.
Since another common wisdom is that factoring is a hard problem, Z3 should not be able to quickly solve instances generated in this way.

Natural numbers are encoded in binary using the *least significant bit* (LSB) convention;
for instance $[1101]_2 = 2^0 + 2^1 + 2^3 = 11.$
We split the task in a number of subproblems.

1. Write a routine $\verb+adder(x, y, z)+$
that, given vectors of variables $x, y$ of length $m$,
and $z$ of length $m + 1$,
returns a formula $\varphi_+$ s.t.
$$ \models \varphi_+(x, y, z) \quad \textrm { iff } \quad  [z]_2 = [x]_2 + [y]_2. $$
*Hint:* Introduce the carry bits $c$ of length $m + 1$:
$$\begin{aligned}
    z[m + 1] &= c[m], \\
    z[i] &= x[i] \oplus y[i] \oplus c[i]
        &&\textrm { for } 1 \leq i \leq m, \textrm { where } \\
    c[1] &= 0, \\
    c[i] &=
        \bigvee_{1 \leq j < i} \left( x[j] \wedge y[j] \wedge
        \bigwedge_{1 \leq j < k < i} x[k] \vee y[k] \right)
        &&\textrm { for } 2 \leq i \leq m.
\end{aligned}$$

2. Write a routine $\verb+multiplier(x, y, z)+$
that, given vectors of variables $x, y$ of length $m$
and $z$ of length $2m$,
returns a formula $\varphi_*$ s.t.
$$ \models \varphi_*(x, y, z) \quad \textrm { iff } \quad  [z]_2 = [x]_2 * [y]_2. $$
*Hint:* Use $\verb+adder(x, y, z)+$.
We define a family $(t_i)_i$ of vectors of variables for $0 \leq i \leq m$
s.t. $t_i$ has size $m + i$,
$t_0 = 0^m$, $z = t_m$,
and, for every $1 \leq i \leq m$,
$$
\begin{aligned}
    &\models \varphi_+(t_{i-1}, x 0^{i-1}, t_i) \quad & \textrm { if } y[i] = 1 \\
    &t_i = 0 t_{i-1} & \textrm{ otherwise}.
\end{aligned}$$

3. Run Z3 on the instance
$$ \exists \bar x, \bar y \cdot \varphi_*(\bar x, \bar y, \hat z) $$
where $[\hat z]_2 = p * q$ with $p = 900004199$, and $q = 900004201$.

In [6]:
from random import randint
import time

# Recall that Z3 supports binary combinators Or, And, Xor; unary Not.
# Or and And can also take a list of formulas.
# Biconditional "<--->" is represented by "==".
# For instance the formula "z <---> x xor y xor c" becomes
# z == Xor(x, Xor(y, c))

# Below there are some useful functions to test the code

# return the number of bits required to represent n in binary
def bits(n):
    return 0 if n == 0 else 1 + bits(n >> 1)

# returns an array of n variables
def mkVars(name, n):
    return [Bool(name + str(k)) for k in range(n)]

# convert an integer into a list of bits (as True, False)
def binary(n, length):
    if length == 0:
        return []
    
    return [True if n & 1 == 1 else False] + binary(n >> 1, length - 1)

# convert a truth assignment back into a number
def number(rho, x):
    return sum([2**i if rho[x[i]] else 0 for i in range(len(x))])

# procedure that tests the adder
def test_adder():
    a = randint(0, 10**10)
    b = randint(0, 10**10)
    
    m = max(bits(a), bits(b))

    x, y, z = mkVars("x", m), mkVars("y", m), binary(a + b, m + 1)

    phi = adder(x, y, z)
    rho = model(phi)

    a1 = number(rho, x)
    b1 = number(rho, y)
    
    print(a1)
    print(b1)
    
    assert(a1 + b1 == a + b)
    print("Adder test passed!")

# uses multiplier to find a factorisation
def factorise(a):
    m = bits(a)

    x, y, z = mkVars("x", m), mkVars("y", m), binary(a, 2 * m)

    t0 = time.time()
    phi = multiplier(x, y, z)
    t1 = time.time()
    
    # exclude trivial factorisations
    phi1 = And(phi, Or([x[i] for i in range(1,m)]), Or([y[i] for i in range(1,m)]))
    print("Instance generated in " + str(t1 - t0) + "s")
    
    rho = model(phi1)
    assert(len(rho) > 0)

    p = number(rho, x)
    q = number(rho, y)
    
    print(p)
    print(q)

    assert(p * q == a)
    return (p, q)

# procedure that tests multiplier
def test_multiplier():
    a = randint(2, 10**4)
    b = randint(2, 10**4)
    
    (a1, b1) = factorise(a * b)
    print("Multiplier test passed!")